In [1]:
pip install awswrangler

ERROR: Could not find a version that satisfies the requirement awswrangler (from versions: none)
ERROR: No matching distribution found for awswrangler
Note: you may need to restart the kernel to use updated packages.


In [2]:
import awswrangler as wr

In [3]:
mt_agg_sample_pd = wr.athena.read_sql_query("SELECT  *  FROM maystreet_mtnorm.mt_aggregated_price_update LIMIT 100", database="maystreet_mtnorm")

In [4]:
mt_agg_sample_pd.columns

Index(['feed', 'firstsequencenumber', 'lastsequencenumber',
       'firstproductsequencenumber', 'lastproductsequencenumber',
       'marketparticipant', 'product', 'firstreceipttimestamp',
       'lastreceipttimestamp', 'firstexchangetimestamp',
       'lastexchangetimestamp', 'firstexchangesendtimestamp',
       'lastexchangesendtimestamp', 'depthofbooklevel', 'biddepthofbooklevel',
       'askdepthofbooklevel', 'isbidupdated', 'isaskupdated', 'tradingstatus',
       'bookstatus', 'bidcustomerquantity', 'askcustomerquantity',
       'bidprice_1', 'bidquantity_1', 'bidnumorders_1', 'bidprice_2',
       'bidquantity_2', 'bidnumorders_2', 'bidprice_3', 'bidquantity_3',
       'bidnumorders_3', 'bidprice_4', 'bidquantity_4', 'bidnumorders_4',
       'bidprice_5', 'bidquantity_5', 'bidnumorders_5', 'bidprice_6',
       'bidquantity_6', 'bidnumorders_6', 'bidprice_7', 'bidquantity_7',
       'bidnumorders_7', 'bidprice_8', 'bidquantity_8', 'bidnumorders_8',
       'bidprice_9', 'bidquantit

In [5]:

for l_chunk in [('a','e'),('e','j'),('j','o'),('o','t'),('t','z')]:
    l_s=l_chunk[0]
    l_e=l_chunk[1]
    print(l_s,l_e)
    wr.athena.read_sql_query(f"select *  from (SELECT  feed,product,count(*) data_count  FROM maystreet_mtnorm.mt_aggregated_price_update \
                             group by feed,product order by feed,product ) t \
                             where lower(feed) >= '{l_s}' and lower(feed) < '{l_e}' ", database="maystreet_mtnorm").\
    to_csv(f's3://maystreetdata/analysis/agg_feed_exch_product_{l_s}_{l_e}.csv')

a e


KeyboardInterrupt: 

In [ ]:
'sa'>'s'

In [ ]:
wr.athena.read_sql_query("select *  from (SELECT  feed,product,count(*) data_count  FROM maystreet_mtnorm.mt_aggregated_price_update \
                         group by feed,product order by feed,product ) t \
                         where lower(feed) >= 'a' and lower(feed) < 'e' ", database="maystreet_mtnorm")

In [ ]:
sql_stmt=f"SELECT *,CASE WHEN side IN ('Ask', 'rest', 'rstd')\
                 THEN 1*quantity ELSE -1*quantity\
            END quantity_signed FROM maystreet_mtnorm.mt_trade where side is not null and product ='{'AMZN'}' order by receipttimestamp desc"
single_name_flow=wr.athena.read_sql_query(sql_stmt, database="maystreet_mtnorm")
single_name_flow=single_name_flow.set_index('receipttimestamp').sort_index()
single_name_flow['price']=single_name_flow['price'].astype(float)
single_name_flow['quantity_signed']=single_name_flow['quantity_signed'].astype(float)

In [ ]:
single_name_flow['price_scaled']=single_name_flow['price']
    
single_name_flow['vol_change']=(single_name_flow['quantity_signed']*single_name_flow['price']).rolling(1000).sum()
#single_name_flow['price']=single_name_flow['price']*10000
single_name_flow.plot.scatter(x='vol_change',y='price_scaled')



In [ ]:
import pandas as pd
import numpy as np
import statsmodels.tsa.stattools as smt

for i in [1,2,3,5,6,7,8,9,10,100,500,1000]:
    #single_name_flow[f'price_scaled_plus_{i}']=single_name_flow['price'].shift(i)
    single_name_flow[f'price_scaled_minus_{i}']=single_name_flow['price'].shift(-i)

In [ ]:
cols = [i for i in single_name_flow.columns if 'price_scaled' in i]+['vol_change']
single_name_flow[cols].describe()

In [ ]:
single_name_flow[cols].corr()

In [6]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hashlib
import pyarrow.parquet as pq
import pyarrow as pa
import s3fs
import boto3
import time
from boto3.dynamodb.conditions import Key
from copy import deepcopy
import awswrangler as wr
#import seaborn as sns
import itertools
import datetime
import math
#import asynch,asyncio

In [ ]:
fs_s3 = s3fs.S3FileSystem(client_kwargs={'region_name': 'us-east-2'})
 
file_name='s3://maystreetdata/feeds_norm/mstnorm_parquet_0_5_0/mt=aggregated_price_update/ex=amex/f=xdp_american_integrated/y=2022/m=07/d=05/dt=2022-07-05/0_1.parquet'
pf = pq.ParquetDataset(file_name,filesystem=fs_s3)
res_pd = pf.read().to_pandas()

In [ ]:
res_pd